<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/book_processing_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install openai
!pip install getpass
!pip install pypdf

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.2 MB/s eta 0:00:00


In [8]:
import openai
from openai import OpenAI
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

client = OpenAI()

In [19]:
from pypdf import PdfReader

reader = PdfReader("taboo.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
#text = page.extract_text()

AttributeError: 'PdfReader' object has no attribute 'extract_text'

In [26]:
def chunk_text(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a document processor used to create fine-tuning data. You will receive a page of a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed. "},
                {"role": "user", "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question. :\n\n{text}"}
                ]
        )
        text_chunks = response.choices[0].message.content.strip().split('\n')
        return text_chunks
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [ ]:
training_json = []
for i in range(10):
  extracted_text = reader.pages[i].extract_text()
  processed_text = chunk_text(extracted_text)
  for chunk in processed_text:
    json_obj = {
                "instruction" : "",
                "input" : "",
                "output" : chunk
            }
    training_json.append(json_obj)

print(json.dumps(training_json, indent=4))

['text ', ' text ', '']
